In [1]:
import cvxpy as cp
import numpy as np

In [3]:
def find_covariance_matrix_m(S_target_np, P_matrices_np, M_dim, solver_name='SCS'):
   
    # --- Input Validation ---
    if not isinstance(S_target_np, np.ndarray) or S_target_np.shape[0] != S_target_np.shape[1]:
        raise ValueError("S_target_np must be a square NumPy array.")
    if S_target_np.shape[0] != M_dim:
        raise ValueError(f"S_target_np dimension ({S_target_np.shape[0]}) must match M_dim ({M_dim}).")
    if not isinstance(P_matrices_np, list) or not all(isinstance(p, np.ndarray) for p in P_matrices_np):
        raise ValueError("P_matrices_np must be a list of NumPy arrays.")
    if not all(p.shape == (M_dim, M_dim) for p in P_matrices_np):
        raise ValueError(f"All permutation matrices in P_matrices_np must be of shape ({M_dim}, {M_dim}).")
    if not P_matrices_np:
        raise ValueError("P_matrices_np cannot be empty.")

    # --- CVXPY Problem Setup ---

    # 1. Define M as a symmetric variable (this implicitly handles M = M.T)
    M = cp.Variable((M_dim, M_dim), symmetric=True)

    # 2. Construct the sum term: sum(P_i.T @ M @ P_i)
    # cvxpy supports the @ operator for matrix multiplication with variables
    sum_term = 0
    for P_i in P_matrices_np:
        sum_term += P_i.T @ M @ P_i

    # 3. Define the objective function: Minimize the Frobenius norm of the difference
    # This allows for cases where S_target might not be perfectly achievable.
    objective = cp.Minimize(cp.norm(sum_term - S_target_np, 'fro'))

    # 4. Define constraints: M must be Positive Semidefinite (PSD)
    # M >> 0 is the cvxpy syntax for M being PSD
    constraints = [M >> 0]

    # 5. Create the problem instance
    problem = cp.Problem(objective, constraints)

    # --- Solve the Problem ---
    try:
        problem.solve(solver=solver_name)
    except cp.SolverError as e:
        print(f"SolverError: {e}. The selected solver '{solver_name}' might not be installed or available for this problem type.")
        print("Try installing other compatible solvers (e.g., pip install Mosek or pip install gurobipy) or explicitly specifying 'SCS'.")
        return None, None, "SolverError"
    except Exception as e:
        print(f"An unexpected error occurred during solving: {e}")
        return None, None, "Error"

    # --- Return Results ---
    if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        # problem.value is the optimal value of the objective function
        # M.value is the NumPy array for the optimized matrix M
        return M.value, problem.value, problem.status
    else:
        print(f"Problem did not solve to optimality. Status: {problem.status}")
        print("This could mean the problem is infeasible, unbounded, or the solver failed to converge.")
        return None, problem.value, problem.status


In [41]:
#np.random.seed(100) # for reproducibility

M_dim = 7 # Increased dimension of the square matrix M

# 1. Define a 'true' M (which is PSD) to simulate the scenario
# Create a random matrix A and then M = A @ A.T to ensure M is PSD
random_matrix_for_M = np.random.randn(M_dim, M_dim)
true_M_np = random_matrix_for_M @ random_matrix_for_M.T
'''
# Add a small diagonal component to ensure it's strictly positive definite,
# helping numerical stability, though not strictly required for PSD.
true_M_np += np.eye(M_dim) * 0.1
'''
print(f"True M (for demonstration, dimension {M_dim}x{M_dim}):\n{np.round(true_M_np, 4)}")


# 2. Define a larger set of 'trickier' permutation matrices (P_i)
# Generate random permutation matrices
num_permutations = 7 # Number of permutation matrices
list_of_P_matrices_np = []
for _ in range(num_permutations):
    # np.random.permutation creates a random permutation of indices
    # np.eye(M_dim)[indices] creates a permutation matrix
    permutation_indices = np.random.permutation(M_dim)
    P_i = np.eye(M_dim)[permutation_indices]
    list_of_P_matrices_np.append(P_i)

print(f"\nUsing {len(list_of_P_matrices_np)} permutation matrices (randomly generated).")
print(list_of_P_matrices_np)

# 3. Generate the target S matrix using the 'true' M and P_i's
# In a real problem, S_target_np would be your given input.
S_target_generated_np = np.zeros((M_dim, M_dim))
for P_i in list_of_P_matrices_np:
    S_target_generated_np += P_i.T @ true_M_np @ P_i

#print(f"\nGenerated S (Target for solver):\n{S_target_generated_np}")

# 4. Call the solver function to find M
found_M_np, optimal_value, status = find_covariance_matrix_m(
        S_target_generated_np, list_of_P_matrices_np, M_dim
    )


if found_M_np is not None:
     print(f"\n--- Solver Results ---")
     print(f"Found M by solver:\n{found_M_np}")
     print(f"Optimal objective value (Frobenius norm of difference): {optimal_value:.4e}")
     print(f"Solver status: {status}")

     # Verify the solution properties
     '''
     reconstructed_S_np = np.zeros((M_dim, M_dim))
     for P_i in list_of_P_matrices_np:
        reconstructed_S_np += P_i.T @ found_M_np @ P_i
     print(f"\nReconstructed S using found M:\n{reconstructed_S_np}")'''

     # Check if the found M is close to the true M (if S was perfectly generated and M is unique)
     print(f"Is found M close to true M (np.allclose)? {np.allclose(found_M_np, true_M_np, atol=1e-6)}")
     print(f"Max abs difference between found M and true M: {np.max(np.abs(found_M_np - true_M_np)):.4e}")

     # Verify if found M is symmetric and PSD
     is_symmetric = np.allclose(found_M_np, found_M_np.T)
     print(f"Is found M symmetric? {is_symmetric}")
     found_eigenvalues = np.linalg.eigvals(found_M_np)
     is_psd = np.all(found_eigenvalues >= -1e-6) # Allow small negative for numerical precision
     print(f"Is found M PSD? {is_psd}. Eigenvalues: {found_eigenvalues}")
else:
     print("\nCould not find M.")



True M (for demonstration, dimension 7x7):
[[ 8.8909 -4.401   0.0401  1.4744 -0.2193  1.2274  3.0822]
 [-4.401  21.9645  2.91    1.52    4.0559 -2.0214 -0.1055]
 [ 0.0401  2.91    9.0071  0.2844 -5.7252  1.4138 -2.0498]
 [ 1.4744  1.52    0.2844  2.9385  0.1584  0.2061  1.445 ]
 [-0.2193  4.0559 -5.7252  0.1584  8.5525 -1.599   3.2139]
 [ 1.2274 -2.0214  1.4138  0.2061 -1.599   3.5746  0.9   ]
 [ 3.0822 -0.1055 -2.0498  1.445   3.2139  0.9     2.9936]]

Using 7 permutation matrices (randomly generated).
[array([[0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]]), array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       